In [3]:
#!/usr/bin/env python
# coding=utf-8
# import ipdb
import argparse
import os
import random
from dataclasses import dataclass
from itertools import chain

import datasets
import evaluate
import torch
from accelerate import Accelerator
from accelerate.utils import set_seed
from accelerate import notebook_launcher
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import transformers
from transformers import (
    AutoConfig,
    AutoTokenizer,
    MT5Model
)

In [4]:
# Global variables
USE_NOTEBOOK_LAUNCHER = False
str_args = None

In [5]:
# Comment out when using .py file
str_args = [
    "--train_file", "./data/train.jsonl",
    "--output_dir", "./output"
]

In [6]:
def parse_args(str_args = None):
    parser = argparse.ArgumentParser()
    parser.add_argument("--train_file", type=str)
    parser.add_argument("--validation_file", type=str)
    parser.add_argument(
        "--gradient_accumulation_steps",
        type=int,
        default=1,
    )
    parser.add_argument(
        "--batch_size",
        type=int,
        default=8,
    )
    parser.add_argument(
        "--learning_rate",
        type=float,
        default=5e-5,
    )
    parser.add_argument(
        "--num_train_epochs",
        type=int,
        default=1,
    )
    parser.add_argument(
        "--output_dir", 
        type=str, 
        default="./output"
    )
    parser.add_argument("--seed", type=int, default=None)
    parser.add_argument("--checkpoint_step", type=str, default=None)

    
    args = parser.parse_args(str_args)
    return args

In [7]:
def main(str_args = None):
    args = parse_args(str_args)
    
    model_name = "google/mt5-small"
    # Prepare 
    if args.seed is not None:
        set_seed(args.seed)
    if args.output_dir is not None:
        os.makedirs(args.output_dir, exist_ok=True)
    
        
    # Initialize accelerator
    accelerator = Accelerator(gradient_accumulation_steps=args.gradient_accumulation_steps)
    
    # Load Dataset
    data_files = {}
    if args.train_file is not None:
        data_files["train"] = args.train_file
    if args.validation_file is not None:
        data_files["validation"] = args.validation_file
     
    # Load Model
    config = AutoConfig.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = MT5Model.from_pretrained(model_name,config=config)

    starting_epoch = 0
    for epoch in range(starting_epoch, args.num_train_epochs):
        model.train()
    

In [8]:
if __name__ == "__main__":
    if USE_NOTEBOOK_LAUNCHER:
        notebook_launcher(main,(str_args,), num_processes=1)
    else:      
        main(str_args)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
d:\anaconda3\envs\ADL2\lib\site-packages\transformers\convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [9]:
from twrouge import get_rouge

TypeError: expected str, bytes or os.PathLike object, not NoneType